In [ ]:
!pip install torchinfo

In [ ]:
!git clone https://github.com/anantgupta129/ERA-V1.git

In [ ]:
# !rm -rf /content/ERA-V1
%cd ERA-V1/session9

In [ ]:
import torch
import torch.optim as optim
from dataset import build_cifar
from models import Net
from torchvision import transforms as T
from utils import Trainer, per_class_accuracy, plot_misclassified, plot_sampledata

# Load Data

In [ ]:
batch_size = 128

kwargs = {
    "batch_size": batch_size,
    "shuffle": True,
    "num_workers": 2,
    "pin_memory": True,
}

train_loader = build_cifar(set="train", **kwargs)
test_loader = build_cifar(set="test", **kwargs)

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)

In [ ]:
model.summarize(device, (1, 3, 32, 32))

In [ ]:
num_epochs = 100
model = Net(drop=0.00).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=0.1, epochs=num_epochs, steps_per_epoch=len(train_loader)
)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1, verbose=True)
# scheduler = None

trainer = Trainer(model, device, optimizer, scheduler)
for epoch in range(1, num_epochs + 1):
    print(f"Epoch {epoch}")
    trainer.train(train_loader)
    trainer.test(test_loader)
    # scheduler.step()

In [ ]:
trainer.plot_history()

In [ ]:
from dataset import make_transform

test_transforms = make_transform("test")
plot_misclassified(model, test_loader, device, test_transforms)

In [ ]:
per_class_accuracy(model, device, test_loader)